In [19]:
!git clone https://github.com/wangluheng328/DS301-NLP-Project.git

Cloning into 'DS301-NLP-Project'...
remote: Enumerating objects: 24677, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 24677 (delta 2), reused 7 (delta 0), pack-reused 24668
Receiving objects: 100% (24677/24677), 215.94 MiB | 20.17 MiB/s, done.
Resolving deltas: 100% (8826/8826), done.
Checking out files: 100% (2582/2582), done.


In [20]:
import glob
import os

root_path = os.path.expanduser('/content/DS301-NLP-Project/scriptbase_alpha')
paths = glob.glob(root_path + '/*')

In [21]:
import pandas as pd

df = pd.DataFrame([], columns = ['title', 'text', 'imdb'])


for path in paths:
  f = open(path + '/script.txt', 'r')
  try:
    text = f.read()
    imdb_dir = path + '/processed/imdb_meta.txt'
    imdb_f = open(imdb_dir, 'r')
    imdb_text = imdb_f.readlines()
    imdb_score = float(imdb_text[1].split('\t')[1].strip())
    df = df.append(pd.Series([path.split('/')[-1], text, imdb_score], index = ['title', 'text', 'imdb']), ignore_index = True)
  except:
    print('Due to decoding issues, ommited: ' + path)

Due to decoding issues, ommited: /content/DS301-NLP-Project/scriptbase_alpha/The Jolson Story
Due to decoding issues, ommited: /content/DS301-NLP-Project/scriptbase_alpha/Goldfinger (film)
Due to decoding issues, ommited: /content/DS301-NLP-Project/scriptbase_alpha/Stranglehold (1931 film)
Due to decoding issues, ommited: /content/DS301-NLP-Project/scriptbase_alpha/Boy Who Never Slept
Due to decoding issues, ommited: /content/DS301-NLP-Project/scriptbase_alpha/From Russia with Love (film)


In [ ]:
df.head()

,title,text,imdb
0,Highlander_ Endgame,"""HIGHLANDER IV: WORLD WITHOUT END"" -- ...",4.5
1,Assassins (film),Assassins (1995) movie script - Screenplays fo...,6.2
2,G.I. Joe_ The Rise of Cobra,\n \n G.I. JOE: THE R...,5.8
3,Supergirl (film),"Revised Screenplay \n17th January, 1983.\n\nAL...",4.3
4,Bottle Rocket,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,7.2


In [31]:
import nltk
from nltk.text import Text
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

nltk.download('wordnet')
nltk.download('stopwords')
stopwords = stopwords.words('english')
stopwords.remove('not')
stopwords.append('\n')
stopwords.append('<b>')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
def process_text(text):
    clean_text = text
    clean_text = clean_text.lower()
    tokens = word_tokenize(clean_text)

    clean_tokens = []

    for token in tokens:
      token = lemmatizer.lemmatize(token)
      if token not in stopwords and token.isalpha():
        clean_tokens.append(lemmatizer.lemmatize(token))
    return clean_tokens

In [33]:
processed = []
for i in tqdm(range(len(df['text']))):
  processed.append(process_text(df['text'][i]))
df['text_preprocessed'] = processed

100%|██████████| 1271/1271 [12:09<00:00,  1.74it/s]


In [40]:
from collections import Counter

count_list = []
for l in df.text_preprocessed:
  count_list += l
top_10k_count = Counter(count_list).most_common(10000)
top_10k = [i[0] for i in top_10k_count]

In [42]:
top_10k[:10]

['look', 'back', 'int', 'not', 'get', 'see', 'one', 'wa', 'know', 'like']

In [46]:
dicts = []
for processed_list in df.text_preprocessed:
  dicts.append(dict(Counter(processed_list)))
df['counts'] = dicts
df.head()

,title,text,imdb,text_preprocessed,counts
0,Highlander_ Endgame,"""HIGHLANDER IV: WORLD WITHOUT END"" -- ...",4.5,"[highlander, iv, world, without, end, draft, b...","{'highlander': 5, 'iv': 2, 'world': 10, 'witho..."
1,Assassins (film),Assassins (1995) movie script - Screenplays fo...,6.2,"[assassin, movie, script, screenplay, var, gaj...","{'assassin': 3, 'movie': 6, 'script': 5, 'scre..."
2,G.I. Joe_ The Rise of Cobra,\n \n G.I. JOE: THE R...,5.8,"[joe, rise, cobra, written, stuart, beattie, j...","{'joe': 15, 'rise': 6, 'cobra': 7, 'written': ..."
3,Supergirl (film),"Revised Screenplay \n17th January, 1983.\n\nAL...",4.3,"[revised, screenplay, january, alexander, salk...","{'revised': 1, 'screenplay': 1, 'january': 1, ..."
4,Bottle Rocket,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,7.2,"[b, window, top, b, bottle, rocket, wes, ander...","{'b': 2, 'window': 25, 'top': 4, 'bottle': 8, ..."


In [49]:
appeared_in = {}
for word in top_10k:
  count = 0
  for dic in df['counts']:
    if word in dic:
      count += 1
  appeared_in[word] = count

In [59]:
import math

tfidf = []
for dic in df.counts:
  # 10k-dimesional tfidf list for the current document
  tfidf_list = []
  for word in top_10k:
    tf = dic.get(word, 0)
    idf = math.log(len(df) / appeared_in[word])
    tfidf_list.append(tf * idf)
  tfidf.append(tfidf_list)

df['tfidf'] = tfidf

In [63]:
df.head()

,title,text,imdb,text_preprocessed,counts,tfidf
0,Highlander_ Endgame,"""HIGHLANDER IV: WORLD WITHOUT END"" -- ...",4.5,"[highlander, iv, world, without, end, draft, b...","{'highlander': 5, 'iv': 2, 'world': 10, 'witho...","[0.028335302525412092, 0.0, 4.6795863553116135..."
1,Assassins (film),Assassins (1995) movie script - Screenplays fo...,6.2,"[assassin, movie, script, screenplay, var, gaj...","{'assassin': 3, 'movie': 6, 'script': 5, 'scre...","[0.10232192578621033, 0.0, 20.615475024751163,..."
2,G.I. Joe_ The Rise of Cobra,\n \n G.I. JOE: THE R...,5.8,"[joe, rise, cobra, written, stuart, beattie, j...","{'joe': 15, 'rise': 6, 'cobra': 7, 'written': ...","[0.08658009104987029, 0.0, 14.291709679735469,..."
3,Supergirl (film),"Revised Screenplay \n17th January, 1983.\n\nAL...",4.3,"[revised, screenplay, january, alexander, salk...","{'revised': 1, 'screenplay': 1, 'january': 1, ...","[0.07870917368170026, 0.0, 10.750401086526681,..."
4,Bottle Rocket,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,7.2,"[b, window, top, b, bottle, rocket, wes, ander...","{'b': 2, 'window': 25, 'top': 4, 'bottle': 8, ...","[0.12829595310117142, 0.0, 5.311962889813183, ..."


In [86]:
imdb_level = []
for i in df.imdb:
  if i >= 6:
    imdb_level.append(3)
  elif i >= 3:
    imdb_level.append(2)
  else:
    imdb_level.append(1)

In [87]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


X_train, X_test, y_train, y_test = train_test_split(tfidf, imdb_level, test_size = 0.3)

In [88]:
lr = LogisticRegression().fit(X_train, y_train)
preds = lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [89]:
sum(preds == y_test)/len(preds)

0.7643979057591623